# DIRECT NORMALIZATION

# 1. GHI

In [12]:
import numpy as np
import rasterio
import os

# ==========================================
#FILE PATHS
# ==========================================
INPUT_RASTER = r"D:\WiDS\Project\Rasters\GHI_TG_100m.tif"
OUTPUT_RASTER = r"D:\WiDS\Project\samp\norm_GHI.tif"
# ==========================================

def normalize_raster(input_path, output_path):
    if not os.path.exists(input_path):
        print(f"Error: The file {input_path} was not found.")
        return

    with rasterio.open(input_path) as src:
        # Read the first band
        data = src.read(1).astype('float32')
        nodata = src.nodata
        
        # Create a mask for valid data
        mask = (data != nodata) if nodata is not None else np.ones(data.shape, dtype=bool)

        # Calculate statistics only on valid pixels
        valid_data = data[mask]
        d_min = np.min(valid_data)
        d_max = np.max(valid_data)

        print(f"Normalizing: Min={d_min}, Max={d_max}")

        # output Initialization array with the nodata value
        normalized = np.full(data.shape, -9999, dtype='float32')

        # Min-Max Calculation: (x - min) / (max - min)
        if d_max - d_min != 0:
            normalized[mask] = (data[mask] - d_min) / (d_max - d_min)
        else:
            normalized[mask] = 0

        # Update metadata for output
        meta = src.meta.copy()
        meta.update({
            "driver": "GTiff",
            "dtype": "float32",
            "nodata": -9999
        })

        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(normalized, 1)
            
    print(f"Success! Saved to: {output_path}")

if __name__ == "__main__":
    normalize_raster(INPUT_RASTER, OUTPUT_RASTER)

Normalizing: Min=4.927274227142334, Max=5.299319744110107
Success! Saved to: D:\WiDS\Project\samp\norm_GHI.tif


# 2. Settlement proximity

In [1]:
import numpy as np
import rasterio
import os

# ==========================================
#FILE PATHS
# ==========================================
INPUT_RASTER = r"D:\WiDS\Project\Rasters\SettlementsProximity_TG_100m.tif"
OUTPUT_RASTER = r"D:\WiDS\Project\samp\norm_SettlementProximity.tif"
# ==========================================

def normalize_raster(input_path, output_path):
    if not os.path.exists(input_path):
        print(f"Error: The file {input_path} was not found.")
        return

    with rasterio.open(input_path) as src:
        # Read the first band
        data = src.read(1).astype('float32')
        nodata = src.nodata
        
        # Create a mask for valid data
        mask = (data != nodata) if nodata is not None else np.ones(data.shape, dtype=bool)

        # Calculate statistics only on valid pixels
        valid_data = data[mask]
        d_min = np.min(valid_data)
        d_max = np.max(valid_data)

        print(f"Normalizing: Min={d_min}, Max={d_max}")

        # output Initialization array with the nodata value
        normalized = np.full(data.shape, -9999, dtype='float32')

        # Min-Max Calculation: (x - min) / (max - min)
        if d_max - d_min != 0:
            normalized[mask] = (data[mask] - d_min) / (d_max - d_min)
        else:
            normalized[mask] = 0

        # Update metadata for output
        meta = src.meta.copy()
        meta.update({
            "driver": "GTiff",
            "dtype": "float32",
            "nodata": -9999
        })

        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(normalized, 1)
            
    print(f"Success! Saved to: {output_path}")

if __name__ == "__main__":
    normalize_raster(INPUT_RASTER, OUTPUT_RASTER)

Normalizing: Min=0.0, Max=22576.314453125
Success! Saved to: D:\WiDS\Project\samp\norm_SettlementProximity.tif


# INVERSE NORMALIZATION

# 3. Cloud Cover

In [15]:
import numpy as np
import rasterio
import os

# ==========================================
#FILE PATHS
# ==========================================
INPUT_RASTER = r"D:\WiDS\Project\Rasters\CloudCover_TG_100m.tif"
OUTPUT_RASTER = r"D:\WiDS\Project\samp\norm_CloudCover.tif"
# ==========================================

def inverse_normalize_raster(input_path, output_path):
    if not os.path.exists(input_path):
        print(f"Error: {input_path} not found.")
        return

    with rasterio.open(input_path) as src:
        data = src.read(1).astype('float32')
        nodata = src.nodata
        
        # Mask valid data
        mask = (data != nodata) if nodata is not None else np.ones(data.shape, dtype=bool)
        
        valid_data = data[mask]
        d_min = np.min(valid_data)
        d_max = np.max(valid_data)

        # output Initialization
        inverted = np.full(data.shape, -9999, dtype='float32')

        # Inverse Normalization: (max - x) / (max - min)
        # This makes high values -> 0 and low values -> 1
        range_val = d_max - d_min
        if range_val != 0:
            inverted[mask] = (d_max - data[mask]) / range_val
        else:
            inverted[mask] = 0

        # Meta update
        meta = src.meta.copy()
        meta.update({
            "dtype": "float32",
            "nodata": -9999
        })

        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(inverted, 1)

    print(f"Inverse normalization complete.")
    print(f"Original Max ({d_max}) is now ~0 | Original Min ({d_min}) is now ~1")

if __name__ == "__main__":
    inverse_normalize_raster(INPUT_RASTER, OUTPUT_RASTER)

Inverse normalization complete.
Original Max (0.5593146085739136) is now ~0 | Original Min (0.3219074010848999) is now ~1


# 4. Slope

In [2]:
import numpy as np
import rasterio
import os

# ==========================================
#FILE PATHS
# ==========================================
INPUT_RASTER = r"D:\WiDS\Project\Rasters\Slope_TG_100m.tif"
OUTPUT_RASTER = r"D:\WiDS\Project\samp\norm_Slope.tif"
# ==========================================

def inverse_normalize_raster(input_path, output_path):
    if not os.path.exists(input_path):
        print(f"Error: {input_path} not found.")
        return

    with rasterio.open(input_path) as src:
        data = src.read(1).astype('float32')
        nodata = src.nodata
        
        # Mask valid data
        mask = (data != nodata) if nodata is not None else np.ones(data.shape, dtype=bool)
        
        valid_data = data[mask]
        d_min = np.min(valid_data)
        d_max = np.max(valid_data)

        # output Initialization
        inverted = np.full(data.shape, -9999, dtype='float32')

        # Inverse Normalization: (max - x) / (max - min)
        # This makes high values -> 0 and low values -> 1
        range_val = d_max - d_min
        if range_val != 0:
            inverted[mask] = (d_max - data[mask]) / range_val
        else:
            inverted[mask] = 0

        # Meta update
        meta = src.meta.copy()
        meta.update({
            "dtype": "float32",
            "nodata": -9999
        })

        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(inverted, 1)

    print(f"Inverse normalization complete.")
    print(f"Original Max ({d_max}) is now ~0 | Original Min ({d_min}) is now ~1")

if __name__ == "__main__":
    inverse_normalize_raster(INPUT_RASTER, OUTPUT_RASTER)

Inverse normalization complete.
Original Max (69.36753845214844) is now ~0 | Original Min (0.0) is now ~1


# 5. Railway Proximity

In [10]:
import numpy as np
import rasterio
import os

# ==========================================
#FILE PATHS
# ==========================================
INPUT_RASTER = r"D:\WiDS\Project\Rasters\RailwayProximity_TG_100m.tif"
OUTPUT_RASTER = r"D:\WiDS\Project\samp\norm_Railway_Proximity.tif"
# ==========================================

def inverse_normalize_raster(input_path, output_path):
    if not os.path.exists(input_path):
        print(f"Error: {input_path} not found.")
        return

    with rasterio.open(input_path) as src:
        data = src.read(1).astype('float32')
        nodata = src.nodata
        
        # Mask valid data
        mask = (data != nodata) if nodata is not None else np.ones(data.shape, dtype=bool)
        
        valid_data = data[mask]
        d_min = np.min(valid_data)
        d_max = np.max(valid_data)

        # output Initialization
        inverted = np.full(data.shape, -9999, dtype='float32')

        # Inverse Normalization: (max - x) / (max - min)
        # This makes high values -> 0 and low values -> 1
        range_val = d_max - d_min
        if range_val != 0:
            inverted[mask] = (d_max - data[mask]) / range_val
        else:
            inverted[mask] = 0

        # Meta update
        meta = src.meta.copy()
        meta.update({
            "dtype": "float32",
            "nodata": -9999
        })

        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(inverted, 1)

    print(f"Inverse normalization complete.")
    print(f"Original Max ({d_max}) is now ~0 | Original Min ({d_min}) is now ~1")

if __name__ == "__main__":
    inverse_normalize_raster(INPUT_RASTER, OUTPUT_RASTER)

Inverse normalization complete.
Original Max (97232.6015625) is now ~0 | Original Min (0.0) is now ~1


# 6. Road Proximity

In [11]:
import numpy as np
import rasterio
import os

# ==========================================
#FILE PATHS
# ==========================================
INPUT_RASTER = r"D:\WiDS\Project\Rasters\RoadProximity_TG_100m.tif"
OUTPUT_RASTER = r"D:\WiDS\Project\samp\norm_Road_Proximity.tif"
# ==========================================

def inverse_normalize_raster(input_path, output_path):
    if not os.path.exists(input_path):
        print(f"Error: {input_path} not found.")
        return

    with rasterio.open(input_path) as src:
        data = src.read(1).astype('float32')
        nodata = src.nodata
        
        # Mask valid data
        mask = (data != nodata) if nodata is not None else np.ones(data.shape, dtype=bool)
        
        valid_data = data[mask]
        d_min = np.min(valid_data)
        d_max = np.max(valid_data)

        # output Initialization
        inverted = np.full(data.shape, -9999, dtype='float32')

        # Inverse Normalization: (max - x) / (max - min)
        # This makes high values -> 0 and low values -> 1
        range_val = d_max - d_min
        if range_val != 0:
            inverted[mask] = (d_max - data[mask]) / range_val
        else:
            inverted[mask] = 0

        # Meta update
        meta = src.meta.copy()
        meta.update({
            "dtype": "float32",
            "nodata": -9999
        })

        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(inverted, 1)

    print(f"Inverse normalization complete.")
    print(f"Original Max ({d_max}) is now ~0 | Original Min ({d_min}) is now ~1")

if __name__ == "__main__":
    inverse_normalize_raster(INPUT_RASTER, OUTPUT_RASTER)

Inverse normalization complete.
Original Max (26242.712890625) is now ~0 | Original Min (0.0) is now ~1


# 7. Temperature

In [18]:
import numpy as np
import rasterio
import os

# ==========================================
#FILE PATHS
# ==========================================
INPUT_RASTER = r"D:\WiDS\Project\Rasters\Temperature_TG_100m.tif"
OUTPUT_RASTER = r"D:\WiDS\Project\samp\norm_Temperature.tif"
# ==========================================

def inverse_normalize_raster(input_path, output_path):
    if not os.path.exists(input_path):
        print(f"Error: {input_path} not found.")
        return

    with rasterio.open(input_path) as src:
        data = src.read(1).astype('float32')
        nodata = src.nodata
        
        # Mask valid data
        mask = (data != nodata) if nodata is not None else np.ones(data.shape, dtype=bool)
        
        valid_data = data[mask]
        d_min = np.min(valid_data)
        d_max = np.max(valid_data)

        # output Initialization
        inverted = np.full(data.shape, -9999, dtype='float32')

        # Inverse Normalization: (max - x) / (max - min)
        # This makes high values -> 0 and low values -> 1
        range_val = d_max - d_min
        if range_val != 0:
            inverted[mask] = (d_max - data[mask]) / range_val
        else:
            inverted[mask] = 0

        # Meta update
        meta = src.meta.copy()
        meta.update({
            "dtype": "float32",
            "nodata": -9999
        })

        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(inverted, 1)

    print(f"Inverse normalization complete.")
    print(f"Original Max ({d_max}) is now ~0 | Original Min ({d_min}) is now ~1")

if __name__ == "__main__":
    inverse_normalize_raster(INPUT_RASTER, OUTPUT_RASTER)

Inverse normalization complete.
Original Max (28.800243377685547) is now ~0 | Original Min (24.92392349243164) is now ~1


# 8. Rainfall

In [19]:
import numpy as np
import rasterio
import os

# ==========================================
#FILE PATHS
# ==========================================
INPUT_RASTER = r"D:\WiDS\Project\Rasters\Rainfall_TG_100m.tif"
OUTPUT_RASTER = r"D:\WiDS\Project\samp\norm_Rainfall.tif"
# ==========================================

def inverse_normalize_raster(input_path, output_path):
    if not os.path.exists(input_path):
        print(f"Error: {input_path} not found.")
        return

    with rasterio.open(input_path) as src:
        data = src.read(1).astype('float32')
        nodata = src.nodata
        
        # Mask valid data
        mask = (data != nodata) if nodata is not None else np.ones(data.shape, dtype=bool)
        
        valid_data = data[mask]
        d_min = np.min(valid_data)
        d_max = np.max(valid_data)

        # output Initialization
        inverted = np.full(data.shape, -9999, dtype='float32')

        # Inverse Normalization: (max - x) / (max - min)
        # This makes high values -> 0 and low values -> 1
        range_val = d_max - d_min
        if range_val != 0:
            inverted[mask] = (d_max - data[mask]) / range_val
        else:
            inverted[mask] = 0

        # Meta update
        meta = src.meta.copy()
        meta.update({
            "dtype": "float32",
            "nodata": -9999
        })

        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(inverted, 1)

    print(f"Inverse normalization complete.")
    print(f"Original Max ({d_max}) is now ~0 | Original Min ({d_min}) is now ~1")

if __name__ == "__main__":
    inverse_normalize_raster(INPUT_RASTER, OUTPUT_RASTER)

Inverse normalization complete.
Original Max (1799.684814453125) is now ~0 | Original Min (753.8519897460938) is now ~1


# 9. Aspect (Encoding)

In [4]:
import numpy as np
import rasterio
import os

# ==========================================
#FILE PATHS
# ==========================================
INPUT_ASPECT = r"D:\WiDS\Project\Rasters\Aspect_TG_100m.tif"
OUTPUT_ENCODED = r"D:\WiDS\Project\samp\norm_Aspect.tif"
# ==========================================

def encode_aspect_suitability(input_path, output_path):
    if not os.path.exists(input_path):
        print(f"Error: {input_path} not found.")
        return

    with rasterio.open(input_path) as src:
        # Read aspect data (degrees 0-360, -1 for flat)
        aspect = src.read(1).astype('float32')
        nodata = src.nodata

        # 1. Handle NoData first by creating a mask
        valid_mask = (aspect != nodata) if nodata is not None else np.ones(aspect.shape, dtype=bool)

        # 2. Define our Conditions (Bins)
        # Note: North is split between 0-22.5 and 337.5-360
        conditions = [
            (aspect == -1),                                       # Flat
            (aspect > 157.5) & (aspect <= 202.5),                 # South (~180)
            ((aspect > 112.5) & (aspect <= 157.5)) |              # SE (135)
            ((aspect > 202.5) & (aspect <= 247.5)),               # SW (225)
            ((aspect > 67.5) & (aspect <= 112.5)) |               # East (90)
            ((aspect > 247.5) & (aspect <= 292.5)),               # West (270)
            ((aspect > 22.5) & (aspect <= 67.5)) |                # NE (45)
            ((aspect > 292.5) & (aspect <= 337.5)),               # NW (315)
            (aspect >= 0) & (aspect <= 22.5) | (aspect > 337.5)   # North (0/360)
        ]

        # 3. Define the corresponding suitability values
        values = [
            1.0,   # Flat
            1.0,   # South
            0.75,  # SE / SW
            0.5,   # East / West
            0.25,  # NE / NW
            0.0    # North
        ]

        # Apply the encoding
        # default=0 ensures any edge cases get the lowest score
        encoded = np.select(conditions, values, default=0)

        # Re-apply NoData mask so background remains NoData
        final_raster = np.where(valid_mask, encoded, -9999)

        # Update metadata
        meta = src.meta.copy()
        meta.update({
            "dtype": "float32",
            "nodata": -9999
        })

        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(final_raster, 1)

    print(f"Encoding complete! Saved to {output_path}")

if __name__ == "__main__":
    encode_aspect_suitability(INPUT_ASPECT, OUTPUT_ENCODED)

Encoding complete! Saved to D:\WiDS\Project\samp\norm_Aspect.tif


# 10. LULC Encoding

In [2]:
import rasterio
import numpy as np

# -----------------------------
# INPUT / OUTPUT
# -----------------------------
lulc_raster   = r"D:\WiDS\Project\Rasters\LULC_TG_100m.tif"
output_raster = r"D:\WiDS\Project\samp\norm_LULC.tif"

NEW_NODATA = -9999.0   # safe NoData value

# -----------------------------
# READ LULC
# -----------------------------
with rasterio.open(lulc_raster) as src:
    lulc = src.read(1)
    profile = src.profile
    old_nodata = src.nodata

# -----------------------------
# output Initialization WITH NODATA
# -----------------------------
suitability = np.full(lulc.shape, NEW_NODATA, dtype="float32")

# -----------------------------
# RECLASSIFY (ENCODE)
# -----------------------------
suitability[lulc == 1] = 0.05   # Forest / Trees
suitability[lulc == 2] = 0.00   # Water
suitability[lulc == 3] = 0.00   # also Built-up but covered with cloud
suitability[lulc == 4] = 0.20   # Flooded vegetation
suitability[lulc == 5] = 0.90   # Background / Barren
suitability[lulc == 6] = 0.00   # Built-up
suitability[lulc == 7] = 1.00   # Rangeland
suitability[lulc == 8] = 0.40   # Crops

# -----------------------------
# PRESERVE ORIGINAL NODATA
# -----------------------------
if old_nodata is not None:
    suitability[lulc == old_nodata] = NEW_NODATA

# -----------------------------
# WRITE OUTPUT
# -----------------------------
profile.update(
    dtype="float32",
    nodata=NEW_NODATA
)

with rasterio.open(output_raster, "w", **profile) as dst:
    dst.write(suitability, 1)

print("LULC suitability encoded....")



LULC suitability encoded....
